In [28]:
import re
import json
import pickle
import os
import sys
import torch
from transformer_lens import HookedTransformer
import plotly.io as pio
import pandas as pd
from itertools import chain
import plotly.graph_objects as go
import torch.nn.functional as F

pio.renderers.default = "notebook_connected"
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
torch.autograd.set_grad_enabled(False)
torch.set_grad_enabled(False)

sys.path.append('../')  # Add the parent directory to the system path

import utils.haystack_utils as haystack_utils
from sparse_coding.train_autoencoder import AutoEncoder
from utils.autoencoder_utils import custom_forward, AutoEncoderConfig, evaluate_autoencoder_reconstruction, get_encoder_feature_frequencies, load_encoder, generate_with_encoder
import utils.haystack_utils as haystack_utils
from process_tiny_stories_data import load_tinystories_validation_prompts


%reload_ext autoreload
%autoreload 2

In [3]:
# Load global state
model_name = "tiny-stories-2L-33M"
model = HookedTransformer.from_pretrained(
    model_name,
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    device=device,
)

l0_encoder, l0_config = load_encoder('18_morning_sun', model_name, model)
l1_encoder, l1_config = load_encoder('8_deep_brook', model_name, model)
prompts = load_tinystories_validation_prompts()

In [30]:
# For single prompt: compare the derivative of the L1 feature wrt L0 mlp input to layer, to linear map (assuming GELU is identity)

torch.autograd.set_grad_enabled(True)
torch.set_grad_enabled(True)
for param in chain(model.parameters(), l0_encoder.parameters(), l1_encoder.parameters()):
    param.requires_grad = True

prompt = "He had to help Daisy. He was about"

def get_run_encoder_hook(encoder, config):
    def hook(value, hook):
        value = value.squeeze(0)
        _, reconstruct, acts, _, _ = encoder(value)
        hook.ctx['acts'] = acts
        return reconstruct.unsqueeze(0)
    return [(f'blocks.{config.layer}.{config.act_name}', hook)]

def access_grad(value, hook):
    hook.ctx['grad'] = value[0, -1]
access_grads = [('blocks.0.mlp.hook_post', access_grad)]

with model.hooks(fwd_hooks=get_run_encoder_hook(l0_encoder, l0_config) + get_run_encoder_hook(l1_encoder, l1_config), bwd_hooks=access_grads):
    model(prompt)
    acts = model.hook_dict['blocks.1.mlp.hook_post'].ctx['acts']
    
    loss = -acts[-1, 880]
    loss.backward()
    grad = model.hook_dict['blocks.0.mlp.hook_post'].ctx['grad']

In [33]:
with torch.no_grad():
    decoder_grads = (l0_encoder.W_dec @ grad).cpu()

    # Histogram of feature gradients, feature believed to contribute to trigram circuit marked in red
    fig = go.Figure(data=[go.Histogram(x=decoder_grads)])
    fig.add_trace(go.Scatter(x=[decoder_grads[7105], decoder_grads[7105]], y=[0, 500], mode='lines', name='Line', line=dict(color='red')))
    fig.update_layout(
        title='Gradients of decoder features in Layer 0 wrt feature 880 in Layer 1',
        xaxis_title='Gradient',
        yaxis_title='Feature count'
    )
    fig.show()

    # Scatter of feature gradients against cosine sims with layer 1 feature believed to contribute to trigram circuit
    W_out = model.W_out[0]
    W_in = model.W_in[1]
    cosine_sims = F.normalize(l0_encoder.W_dec @ W_out, dim=-1) @ F.normalize(W_in @ l1_encoder.W_enc[:, 880], dim=0)
    # print(cosine_sims[7105])

    fig = go.Figure(data=go.Scatter(x=decoder_grads, y=cosine_sims.cpu(), mode='markers'))
    fig.update_layout(
        title="Gradients of decoder features in Layer 0 wrt a Layer 1 feature against cosine similarities of same",
        xaxis_title='Gradient',
        yaxis_title='Cosine similarity')
    fig.show()

tensor(0.4307, device='cuda:0')


In [7]:
# model wants loss to be lower, model wants acts to be lower, negative gradients say lower early layer acts would cause lower late layer acts
def change_grad(value, hook):
    values, indices = torch.topk(value[0, -1], 10)
    print(indices)
    value[0, -1, indices[0]] -= (grad[indices[0]] * 0.05)
    _, reconstruct, acts, _, _ = l0_encoder(value.squeeze(0))
    acts.retain_grad()
    return reconstruct.unsqueeze(0)
change_grads = [(f'blocks.{l0_config.layer}.{l0_config.act_name}', change_grad)]

with model.hooks(change_grads + get_run_encoder_hook(l1_encoder, l1_config)):
    model(prompt)

tensor(7.4160, device='cuda:0', grad_fn=<SelectBackward0>)
